## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Explain the purpose of this analysis:
find the applicants that will be successful if funded by AlphabetSoup organisation.

Results:
>using ReLU as the neural network activation function in starter_code .
>This function is very popular as it gives result close to 0 or 1.
>It doesnot activate all the neurons at the same time. the neurons will get deactivated when the output of linear transformation is less than 0 and therefore it will ignore negative values.
The sigmoid function:
>This is a non-linear activation function.
>The sigmoid transforms the values in the range of 0 and 1.
> since sigmoid is non linear only when there are multiple neurons with sigmoid function as result of this the output is non linear.
the sigmoid function is not symetric around zero so the output of sigmoid function is of the same sign.
>incontrast the tanh function is symmetric near 0 and so it uses a range from -1 to +1. the tanh function is better than sigmoid as the gradients are not restricted to move in certain direction.

leaky_ReLU(LR):
>LR Is a variation of ReLu, it corrects the disadvantages of ReLU function. in Leaky ReLU(LR) function, we use a small linear component to the values that are less than 0 to activate the neurons.

ELU activation function:
>Exponential linear unit that is unlike the LR and parametric ReLu, as it shows the negative values in the form of a logarithmic curve instead of straight line.

Swish:
>This is a relatively new discovery by google . swish is computationally efficient and shows better performance than ReLU on deeper models. the swish ranges from -infinity to +infinity.

Data Preprocessing:
1. the columns EIN and NAME were dropped.
2. the target for the model is the  the status or the is_successful column
3. the features of the model is the classification, application type, income_bracket, organisation, affiliation, ask_amount, special_consideration.
4. status was not a feature or target.
Compiling, training and evaluating model:
1. we selected 3 activaition layers.
2. when we use only 2 activation functions we can achieve 99% accuracy, whereas when we mix multiple linear and non linear activiation functions the accuracy reduced to 72%
3. increasing the activation layers increased the accuracy of the model performance.



In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
newapplication_df = application_df.drop(columns = ['EIN', 'NAME'])
newapplication_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Determine the number of unique values in each column.
uniquevaluecounts = newapplication_df.nunique()
uniquevaluecounts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
valuecnts = newapplication_df['APPLICATION_TYPE'].value_counts()
valuecnts_df = pd.DataFrame(valuecnts)
valuecnts_df.dtypes
to_replace = []
valcnts_not_reqd= newapplication_df['APPLICATION_TYPE'].value_counts() < 500
valcnts_not_reqd[8:]
valuecnts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ['T9','T12', 'T13', 'T14', 'T25', 'T29', 'T17','T2', 'T15']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
new_classificationcnts = newapplication_df['CLASSIFICATION'].value_counts()
classificationcnts = pd.DataFrame(data=new_classificationcnts)
classificationcnts.reset_index()
newclassificationcnts = classificationcnts.loc[classificationcnts['CLASSIFICATION']>1, :]
newclassificationcnts


,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
newclassificationcnts

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
other_classifications=classificationcnts.loc[classificationcnts['CLASSIFICATION']<1000, 'CLASSIFICATION']
OC_df = pd.DataFrame(other_classifications)
classifications_to_replace.append(OC_df.index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categoryto_numeric_df = pd.get_dummies(newapplication_df)
categoryto_numeric_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = categoryto_numeric_df.drop(columns = 'STATUS')
y = categoryto_numeric_df['STATUS']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify =y)
X_train

#X_test_first40
y_test

26554    1
4919     1
7337     1
8855     1
33414    1
        ..
15581    1
30389    1
7787     1
5966     1
23763    1
Name: STATUS, Length: 8575, dtype: int64

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()
first25724_X_train = X_train[:25724]
# Fit the StandardScaler
X_scaler = scaler.fit(first25724_X_train)

# Scale the data
X_train_scaled = X_scaler.transform(first25724_X_train)
X_test_scaled = X_scaler.transform(first25724_X_train)
print(len(X_train_scaled))
print(len(y_test))
X_train_scaled
y_test

25724
25724


26554    1
4919     1
7337     1
8855     1
33414    1
        ..
15581    1
30389    1
7787     1
5966     1
23763    1
Name: STATUS, Length: 8575, dtype: int64

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=116))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim=80))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="sigmoid", input_dim= 80))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                9360      
                                                                 
 dense_9 (Dense)             (None, 80)                6480      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 18301 (71.49 KB)
Trainable params: 18301 (71.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
train_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=10)
train_model

Epoch 1/100
2573/2573 [==============================] - 12s 4ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 2/100
2573/2573 [==============================] - 8s 3ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 3/100
2573/2573 [==============================] - 6s 2ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 4/100
2573/2573 [==============================] - 5s 2ms/step - loss: 0.0014 - accuracy: 0.9998
Epoch 5/100
2573/2573 [==============================] - 5s 2ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 6/100
2573/2573 [==============================] - 6s 2ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 7/100
2573/2573 [==============================] - 5s 2ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 8/100
2573/2573 [==============================] - 6s 2ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 9/100
2573/2573 [==============================] - 5s 2ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 10/100
2573/2573 [==============================] - 12s 5ms/step -

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled[:8575],y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0012 - accuracy: 0.9999 - 458ms/epoch - 2ms/step
Loss: 0.001173236989416182, Accuracy: 0.9998833537101746


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.keras')